In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split, cross_val_score

In [6]:
input_file = 'income_data.txt'

In [7]:
X = []
y = []
count_class1 = 0
count_class2 = 0
max_datapoints = 25000

In [8]:
with open(input_file, 'r') as f:
    for line in f.readlines():
        if count_class1 >= max_datapoints and count_class2 >= max_datapoints:
            break

        if '?' in line:
            continue

        data = line[:-1].split(',')
        # print(data)

        if data[-1] == ' <=50K' and count_class1 < max_datapoints:
            X.append(data)
            count_class1 += 1

        if data[-1] == ' >50K' and count_class2 < max_datapoints:
            X.append(data)
            count_class2 += 1

In [9]:
X = np.array(X)

In [10]:
label_encoder = []
X_encoded = np.empty(X.shape)
for i, item in enumerate(X[0]):
    if item.isdigit():
        X_encoded[:, i] = X[:, i]
    else:
        label_encoder.append(preprocessing.LabelEncoder())
        X_encoded[:, i] = label_encoder[-1].fit_transform(X[:, i])

X = X_encoded[:, :-1].astype(int)
y = X_encoded[:, -1].astype(int)

In [11]:
classifier = OneVsRestClassifier(LinearSVC(random_state=0))

In [12]:
classifier.fit(X, y)

OneVsRestClassifier(estimator=LinearSVC(random_state=0))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [15]:
classifier = OneVsRestClassifier(LinearSVC(random_state=0))
classifier.fit(X_train, y_train)
y_test_pred = classifier.predict(X_test)

In [16]:
f1 = cross_val_score(classifier, X, y, scoring='f1_weighted', cv=3)
print('F1 score: ' + str(round(100*f1.mean(), 2)) + '%')

F1 score: 74.21%


In [17]:
input_data = ['37', ' Private', ' 215646', ' HS-grad', ' 9', ' Never-married', ' Handlers-cleaners', ' Not-in-family', ' White', ' Male', ' 0', ' 0', ' 40', ' United-States']

In [18]:
input_data_encoded = [-1] * len(input_data)
count = 0
for i, item in enumerate(input_data):
    if item.isdigit():
        input_data_encoded[i] = int(item)
    else:
        input_data_encoded[i] = label_encoder[count].transform([item])[0]
        count += 1

input_data_encoded = np.array(input_data_encoded).reshape(1, -1)

In [19]:
predicted_class = classifier.predict(input_data_encoded)

In [20]:
print(label_encoder[-1].inverse_transform(predicted_class)[0])

 <=50K


In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Оцінка моделі
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')

print(f'Акуратність: {accuracy*100:.2f}%')
print(f'Точність: {precision*100:.2f}%')
print(f'Повнота: {recall*100:.2f}%')
print(f'F1: {f1*100:.2f}%')

Акуратність: 78.14%
Точність: 76.50%
Повнота: 78.14%
F1: 74.32%
